Este notebook foi baseado [neste](https://github.com/PyGoogle/PyD3/blob/master/iris_scatterplot.ipynb) repositório.

# 0. Imports

In [ ]:
from IPython.core.display import HTML
from string import Template
import pandas as pd
import json, random
import ipywidgets as widgets

In [ ]:
HTML('''
<h1>Hello HIAAC!</h1>
<h3>Essa lib manipula o DOM</h3>
''')

A função `IPython.core.display.HTML` é a responsável por renderizar trechos de HTML no notebook e é a nossa ponte para utilizarmos o D3.js no jupyter.

In [ ]:
HTML('<script src="https://d3js.org/d3.v7.min.js"></script>')

ps.: Ao tentar rodar este notebook no jupyter notebook este script não funcionou. Talvez este notebook não funcione dependendo do sistema em que ele roda. Estou utilizando jupiter lab no Google chrome, Windows e Anaconda para gerenciamento de pacotes.

# 1. Carregar dados

In [ ]:
filename = 'web3/data.tsv'
iris = pd.read_csv(filename,sep="\t")
iris.head()

Os dados são obtidos de um arquivo .tsv e convertidos para dicionário num formato que pode ser aceito pelo JSON e apropriado para o scatterplot que desejamos criar.

In [ ]:
iris_array_of_dicts = iris.to_dict(orient='records')
iris_array_of_dicts[:5]

# 2. Script feito por string

Abaixo foram criados templates baseados no explemplo `web3`:

In [ ]:
css_text = '''
.axis path,
.axis line {
  fill: none;
  stroke: #000;
  shape-rendering: crispEdges;
}

.dot {
  stroke: #000;
}
'''

In [ ]:
js_text_template = Template('''
var margin = {top: 20, right: 20, bottom: 30, left: 40},
// ****    width = 960 - margin.left - margin.right, ****
// ****    height = 500 - margin.top - margin.bottom; ****
    width = 720 - margin.left - margin.right,
    height = 375 - margin.top - margin.bottom;

var x = d3.scaleLinear().range([0, width]);

var y = d3.scaleLinear().range([height, 0]);

var color = d3.scaleOrdinal(d3.schemeCategory10);

var xAxis = d3.axisBottom(x);

var yAxis = d3.axisLeft(y);

// **** var svg = d3.select("body").append("svg") ****
var svg = d3.select("#$graphdiv").append("svg")
    .attr("width", width + margin.left + margin.right)
    .attr("height", height + margin.top + margin.bottom)
  .append("g")
    .attr("transform", "translate(" + margin.left + "," + margin.top + ")");

// **** d3.tsv("data.tsv", function(error, data) { ****
// ****  if (error) throw error; ****

var data = $python_data ;

  data.forEach(function(d) {
    d.sepalLength = +d.sepalLength;
    d.sepalWidth = +d.sepalWidth;
  });

  x.domain(d3.extent(data, function(d) { return d.sepalWidth; })).nice();
  y.domain(d3.extent(data, function(d) { return d.sepalLength; })).nice();

  svg.append("g")
      .attr("class", "x axis")
      .attr("transform", "translate(0," + height + ")")
      .call(xAxis)
    .append("text")
      .attr("class", "label")
      .attr("x", width)
      .attr("y", -6)
      .style("text-anchor", "end")
      .text("Sepal Width (cm)");

  svg.append("g")
      .attr("class", "y axis")
      .call(yAxis)
    .append("text")
      .attr("class", "label")
      .attr("transform", "rotate(-90)")
      .attr("y", 6)
      .attr("dy", ".71em")
      .style("text-anchor", "end")
      .text("Sepal Length (cm)")

  svg.selectAll(".dot")
      .data(data)
    .enter().append("circle")
      .attr("class", "dot")
      .attr("r", 3.5)
      .attr("cx", function(d) { return x(d.sepalWidth); })
      .attr("cy", function(d) { return y(d.sepalLength); })
      .style("fill", function(d) { return color(d.species); });

  var legend = svg.selectAll(".legend")
      .data(color.domain())
    .enter().append("g")
      .attr("class", "legend")
      .attr("transform", function(d, i) { return "translate(0," + i * 20 + ")"; });

  legend.append("rect")
      .attr("x", width - 18)
      .attr("width", 18)
      .attr("height", 18)
      .style("fill", color);

  legend.append("text")
      .attr("x", width - 24)
      .attr("y", 9)
      .attr("dy", ".35em")
      .style("text-anchor", "end")
      .text(function(d) { return d; });

// **** }); ****

''')

Abaixo foi criado um template HTML com um elemento `div` onde o gráfico será renderizado. Os dados obtidos do arquivo são convertidos numa string formato JSON e inseritos diretamente no template HTML, fazendo, asssim, a conexão entre a execução em Python e Javascript.

In [ ]:
html_template = Template('''
<style> $css_text </style>
<div id="graph-div"></div>
<script> $js_text </script>
''')
js_text = js_text_template.substitute({'python_data': json.dumps(iris_array_of_dicts),
                                       'graphdiv': 'graph-div'})
HTML(html_template.substitute({'css_text': css_text, 'js_text': js_text}))

# 3. Script por arquivo externo

Aqui o script é gerado à partir do arquivo que se encontra no exemplo `web3`, aqui a ideia é gerar scripts à partir de outros scripts que já funcionam diretamente numa página web normal. Para funcionar foi necessário remover a palavra `export` da função.

In [ ]:
js_text = open("web3/scatterplot.js", "r").read().replace('export ', '')
css_text = open("web3/style.css", "r").read()

html_template = Template('''
<style> $css_text </style>
<div id="graph-div2"></div>
<script>
$js_text

scatterplot($python_data, "#graph-div2");
</script>
''')
HTML(html_template.substitute({'css_text': css_text,
                               'js_text': js_text,
                               'python_data': json.dumps(iris_array_of_dicts)}))

# 4. Exemplo web4

Vários exemplos de gráficos D3.js podem er encontrados neste [link](https://observablehq.com/@d3/gallery).
Algumas adaptações no código são necessárias.

In [ ]:
json_file = open("web4/flare-2.json")
flare_dicts = json.load(json_file)
js_text = open("web4/circle-packing.js", "r").read().replace('export ', '')

html_template = Template('''
<div id="graph-div3"></div>
<script>
$js_text

ciclePack($python_data, "#graph-div3");
</script>
''')
HTML(html_template.substitute({'js_text': js_text,
                               'python_data': json.dumps(flare_dicts)}))

# 5. Widgets do python

Aqui é uma tentativa de integrar as funcionalidades do `ipywidgets` com o D3.js.
Os dados são selecionados utilizando os widgets e à cada mudança nos dados selecionados o HTML é renderizado novamente com os novos dados.

In [ ]:
def generate_scatterplot(htmlElement:str, data):
    print(htmlElement)
    print(len(data))
    js_text = open("web3/scatterplot.js", "r").read().replace('export ', '')
    css_text = open("web3/style.css", "r").read()

    html_template = Template('''
    <style> $css_text </style>
    <div id="$Graphdiv"></div>
    <script>
    $js_text

    scatterplot($python_data, "#$Graphdiv");
    </script>
    ''')
    html_string = html_template.substitute({'Graphdiv': htmlElement,
                                   'css_text': css_text,
                                   'js_text': js_text,
                                   'python_data': json.dumps(data)})
    display(HTML(html_string))

minPetalLength = iris['petalLength'].min()
maxPetalLength = iris['petalLength'].max()
minPetalWidth = iris['petalWidth'].min()
maxPetalWidth = iris['petalWidth'].max()
species = iris['species'].unique().tolist()
species.append('all')

def select_data(rangePetalLength=[minPetalLength, maxPetalLength],
                rangePetalWidth=[minPetalWidth, maxPetalWidth],
                species = 'all'):
    df = iris[(iris['petalLength'] >= rangePetalLength[0]) & (iris['petalLength'] <= rangePetalLength[1])]
    df = df[(df['petalWidth'] >= rangePetalWidth[0]) & (df['petalWidth'] <= rangePetalWidth[1])]
    if(species != 'all'):
        df = df[df['species'] == species]
    data = df.to_dict(orient='records')
    generate_scatterplot('graph-div4', data)

In [ ]:
widgets.interact(select_data,
                 rangePetalLength=widgets.FloatRangeSlider(min=minPetalLength,max=maxPetalLength,step=0.1,
                                                        value=(minPetalLength, maxPetalLength)),
                 rangePetalWidth=widgets.FloatRangeSlider(min=minPetalWidth,max=maxPetalWidth,step=0.1),
                                                        value=(minPetalWidth, maxPetalWidth),
                 species=species)

# 6. Widgets do Javascript

Aqui é um exemplo de como criar widgets utilizando as ferramentas do D3.js e Javascript. O mesmo exemplo também se encontra no `web5`. Aparentemente a animação por este método é mais suave do que utilizando widgets do Python.

In [ ]:
css_text = open("web3/style.css", "r").read()

html_template = Template('''
<style> $css_text </style>
<div>Petal Length Slider</div>
<input type="range" id="petal-length-slider" name="petal-length-slider">
<div id="graph-div5"></div>
<script type="module">
$js_text
function updatePlot(values) {
  d3.select("#graph-div5").selectAll('*').remove()
  scatterplot(values, "#graph-div5");
}

var data = $python_data;
var min = d3.min(data, (d) => d.petalLength);
var max = d3.max(data, (d) => d.petalLength);
var slider = d3.select("#petal-length-slider");

slider.attr("min", min).attr("max", max).attr("step", 0.1).attr("value", max);
slider.on("change", (d) => {
  var values = data.filter(l => l.petalLength <= d.target.value)
  updatePlot(values);
})

scatterplot($python_data, "#graph-div5");
</script>
''')
HTML(html_template.substitute({'css_text': css_text,
                               'js_text': js_text,
                               'python_data': json.dumps(iris_array_of_dicts)}))

# 7. Ipyd3

Ao pesquisar por bibliotecas que integram o D3.js foi encontrado o ipyd3, porém, esta biblioteca não tem muitos recursos além de plotar gráficos de nós e de árvore.

Quando atualizei as libs ele parou de funcionar, não vale a pena.

In [ ]:
 import ipyd3

In [ ]:
graph = {"nodes":
            [
                {"id":"node11", "layer":"layer1", "group":"group1"},
                {"id":"node1", "layer":"layer1", "group":"group1" },
                {"id":"node2", "layer":"layer1", "group":"group1" },
                {"id":"node5", "layer":"layer1" },
                {"id":"node4", "layer":"layer1" },
                {"id":"node3", "layer":"layer1" },
                {"id":"node6", "layer":"layer1" },
                {"id":"node7", "layer":"layer1" },
                {"id":"node8", "layer":"layer1", "group":"group2"},
                {"id":"node9", "layer":"layer1", "group":"group2"},
                {"id":"node10", "layer":"layer1","group":"group2"}
            ],
         "links":
             [
                 {
                     "source":"node1",
                     "target":"node2",
                     "layer":"layer1-layer1"
                 },
                  {
                     "source":"node11",
                     "target":"node2",
                     "layer":"layer1-layer1"
                 },
                 {
                     "source":"node1",
                     "target":"node3",
                     "layer":"layer1-layer1"
                 },
                 {
                     "source":"node1",
                     "target":"node4",
                     "layer":"layer1-layer1"
                 },
                 {
                     "source":"node1",
                     "target":"node5",
                     "layer":"layer1-layer1"
                 },
                 {
                     "source":"node5",
                     "target":"node6",
                     "layer":"layer1-layer1"
                 },
                 {
                     "source":"node4",
                     "target":"node7",
                     "layer":"layer1-layer1"
                 },
                 {
                     "source":"node3",
                     "target":"node8",
                     "layer":"layer1-layer1"
                 },
                 {
                     "source":"node2",
                     "target":"node9",
                     "layer":"layer1-layer1"
                 },
                 {
                     "source":"node9",
                     "target":"node10",
                     "layer":"layer1-layer1"
                 }
             ]
        }
ipyd3.ForceDirectedGraph(graph = graph, width=900)

In [ ]:
data = {
  "name": "flare",
  "children": [
   {
    "name": "analytics",
    "children": [
     {
      "name": "cluster",
      "children": [
       {"name": "AgglomerativeCluster", "value": 3938},
       {"name": "CommunityStructure", "value": 3812},
       {"name": "HierarchicalCluster", "value": 6714},
       {"name": "MergeEdge", "value": 743}
      ]
     },
     {
      "name": "graph",
      "children": [
       {"name": "BetweennessCentrality", "value": 3534},
       {"name": "LinkDistance", "value": 5731},
       {"name": "MaxFlowMinCut", "value": 7840},
       {"name": "ShortestPaths", "value": 5914},
       {"name": "SpanningTree", "value": 3416}
      ]
     },
     {
      "name": "optimization",
      "children": [
       {"name": "AspectRatioBanker", "value": 7074}
      ]
     }
    ]
   }
  ]
 }

In [ ]:
 hg = ipyd3.HierarchicalGraph(data=data)

hg

# 8. Plotly

Plotly é uma biblioteca para geração de gráficos que utiliza o D3.js como base. Vários exemplos de gráficos podem ser encontrados [aqui](https://plotly.com/python/).

In [ ]:
import plotly.express as px 
  
fig = px.scatter(iris, x="sepalWidth", y="sepalLength", color="species",
                 size='petalLength', hover_data=['petalWidth'])
fig.show()

# 9. Dash e Plotly

Dash é um framework para visualização de interfaces que gera páginas web e elementos UI utilizando ferramentas em Python. Neste exemplo é demonstrado como um elemento de slider interage com o gráfico.

In [ ]:
from dash import Dash, dcc, html, Input, Output
import plotly.express as px
from jupyter_dash import JupyterDash

app = JupyterDash(__name__)


app.layout = html.Div([
    html.H4('Interactive scatter plot with Iris dataset'),
    dcc.Graph(id="scatter-plot"),
    html.P("Filter by petal width:"),
    dcc.RangeSlider(
        id='range-slider',
        min=0, max=2.5, step=0.1,
        marks={0: '0', 2.5: '2.5'},
        value=[0.5, 2]
    ),
])


@app.callback(
    Output("scatter-plot", "figure"), 
    Input("range-slider", "value"))
def update_bar_chart(slider_range):
    low, high = slider_range
    mask = (iris['petalWidth'] > low) & (iris['petalWidth'] < high)
    fig = px.scatter(
        iris[mask], x="sepalWidth", y="sepalLength", 
        color="species", size='petalLength', 
        hover_data=['petalWidth'])
    return fig


app.run_server(mode="inline")

# 10. Plotly e ipywidgets

Apesar do Dash já oferecer seu widgets, é possivel utilizar ipywidgets com o Plotly.

In [ ]:
fig = px.scatter(iris, x="sepalWidth", y="sepalLength", color="species",
                 size='petalLength', hover_data=['petalWidth'])

rangePetalWidth = widgets.FloatRangeSlider(min=0,max=2.5,step=0.1)
def response(change):
    mask = (iris['petalWidth'] >= rangePetalWidth.value[0]) & (iris['petalWidth'] <= rangePetalWidth.value[1])
    df = iris[mask]
    fig = px.scatter(df, x="sepalWidth", y="sepalLength", color="species",
                 size='petalLength', hover_data=['petalWidth'])
    fig.show()

out = widgets.interactive_output(response, {'change':rangePetalWidth})
display(rangePetalWidth, out)    

Outro exemplo de como fazer essa interação pode ser encontrado [aqui](https://plotly.com/python/figurewidget-app/).

# 11. Dash e D3

O Dash permite o encapsulamento do D3.js com React podendo criar instâncias de gráficos customizados. Instruções se encontram neste [link](https://dash.plotly.com/d3-react-components).